### Exercise 1: Enhanced ReAct with Reasoning Traces


---
## 🎯 Practice Exercises

### Exercise 1: Enhanced ReAct with Reasoning Traces

**Task:** Modify the ReAct agent to explicitly show its reasoning.

**Requirements:**
1. Add a tool called `think` that logs reasoning
2. Agent must call `think` BEFORE any action tool
3. Store reasoning traces in custom state
4. Display full reasoning chain to user

**Example output:**
```
Thought: I need to calculate 2+2
Action: calculator("2+2")
Observation: 4
Thought: Now I have the answer
Final Answer: The result is 4
```

**Deliverables:**
- Enhanced ReAct agent with thinking
- Test with 3 multi-step queries
- Compare verbosity vs standard ReAct

---



In [ ]:
%pip install -q python-dotenv langchain-openai langgraph

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated, Literal, Sequence
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, ToolMessage, SystemMessage
import operator
import os

# ─── Load API key ─────────────────────────────────────────────────────
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded successfully")

# ─── LLM Setup ────────────────────────────────────────────────────────
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

# ─── Define Tools ─────────────────────────────────────────────────────
@tool
def calculator(expression: str) -> str:
    """Calculate simple math expressions safely"""
    try:
        return str(eval(expression, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Error: {str(e)}"


@tool
def search(query: str) -> str:
    """Dummy search with some hardcoded knowledge"""
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework for building stateful multi-actor applications.",
        "react": "ReAct is an agent pattern that combines reasoning and acting."
    }
    query = query.lower()
    for key, value in knowledge.items():
        if key in query:
            return value
    return "No information found."


@tool
def think(reasoning: str) -> str:
    """Use this tool to explicitly record your step-by-step thinking.
    You MUST call this BEFORE using any other tool or giving final answer."""
    return f"Thought recorded: {reasoning}"


tools = [calculator, search, think]
llm_with_tools = llm.bind_tools(tools)

# ─── Custom State ─────────────────────────────────────────────────────
class ReActTraceState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    reasoning_trace: Annotated[list[str], operator.add]


# ─── Nodes ────────────────────────────────────────────────────────────
def assistant(state: ReActTraceState):
    system_prompt = """You are a thoughtful, step-by-step assistant.
You MUST follow this exact pattern for EVERY response:
1. First call the 'think' tool to record your current reasoning
2. Then (and only then) call other tools if needed or give the final answer

Always be very explicit and detailed in your thinking."""

    messages = [SystemMessage(content=system_prompt)] + state["messages"]

    response: AIMessage = llm_with_tools.invoke(messages)

    # Optional: show thought immediately during execution
    if response.tool_calls:
        for tc in response.tool_calls:
            if tc["name"] == "think":
                thought = tc["args"].get("reasoning", "").strip()
                if thought:
                    print(f"🧠 Explicit thought: {thought}\n")

    return {
        "messages": [response],
        "reasoning_trace": [response.content.strip()]
    }


def should_continue(state: ReActTraceState) -> Literal["tools", "__end__"]:
    last_msg = state["messages"][-1]
    if isinstance(last_msg, AIMessage) and last_msg.tool_calls:
        return "tools"
    return "__end__"


# ─── Build & Compile Graph ────────────────────────────────────────────
builder = StateGraph(ReActTraceState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

react_with_trace = builder.compile(checkpointer=MemorySaver())

print("Enhanced ReAct agent with explicit 'think' tool created ✓")
print("You can now test it with queries like:")
print("  - What is 19 × 23?")
print("  - Who created Python?")
print("  - What happens if I divide 100 by 0?")

Note: you may need to restart the kernel to use updated packages.
✅ API key loaded successfully
✅ LLM initialized: gpt-4o-mini
Enhanced ReAct agent with explicit 'think' tool created ✓
You can now test it with queries like:
  - What is 19 × 23?
  - Who created Python?
  - What happens if I divide 100 by 0?


In [7]:
# Simple test function
def test_agent(query: str):
    print("\n" + "═"*80)
    print(f"QUERY: {query}")
    print("═"*80 + "\n")

    result = react_with_trace.invoke(
        {"messages": [HumanMessage(content=query)], "reasoning_trace": []},
        config={"configurable": {"thread_id": "test"}}
    )

    print("FINAL RESULT:\n")
    for msg in result["messages"]:
        if isinstance(msg, HumanMessage):
            print(f"👤 User: {msg.content}")
        elif isinstance(msg, AIMessage):
            print(f"🤖 Assistant: {msg.content.strip()}")
            if msg.tool_calls:
                print("   Tool calls:", [c["name"] for c in msg.tool_calls])
        elif isinstance(msg, ToolMessage):
            print(f"🛠 {msg.name} → {msg.content}")
        print("─"*70)

    print("\nReasoning trace collected:")
    for trace in result["reasoning_trace"]:
        print("•", trace[:200] + ("..." if len(trace)>200 else ""))
    print("═"*80 + "\n")


# Run some tests
test_agent("What is 17 times 23?")
test_agent("Who created the Python programming language?")
test_agent("Calculate 100 / 0 and explain what happens")


════════════════════════════════════════════════════════════════════════════════
QUERY: What is 17 times 23?
════════════════════════════════════════════════════════════════════════════════

🧠 Explicit thought: To calculate 17 times 23, I need to multiply these two numbers together. This is a straightforward multiplication problem that can be solved using basic arithmetic.

FINAL RESULT:

👤 User: What is 17 times 23?
──────────────────────────────────────────────────────────────────────
🤖 Assistant: 
   Tool calls: ['think']
──────────────────────────────────────────────────────────────────────
🛠 think → Thought recorded: To calculate 17 times 23, I need to multiply these two numbers together. This is a straightforward multiplication problem that can be solved using basic arithmetic.
──────────────────────────────────────────────────────────────────────
🤖 Assistant: 
   Tool calls: ['calculator']
──────────────────────────────────────────────────────────────────────
🛠 calculator → 391

## Exercise 2: Plan-Execute with Replanning

**Task:** Add failure handling and replanning to Plan-Execute.

**Requirements:**
1. Executor can return "FAILED: reason"
2. On failure, call replanner node
3. Replanner creates new plan incorporating failure info
4. Max 1 replan attempt to prevent loops
5. Show both original and revised plans

**Test scenario:**
```
Task: "Calculate 10/0, then explain the result"
Expected: Plan fails → Replans to explain division by zero
```

**Deliverables:**
- Plan-Execute with replanning
- Test with task designed to fail
- Document replanning decision

---



In [ ]:
from typing import List, TypedDict, Annotated
import operator

# Assuming 'llm', 'calculator' and 'search' tools are already defined earlier in your notebook
# If not, you can add them like this (uncomment if needed):
"""
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    try:
        return str(eval(expression, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search(query: str) -> str:
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework...",
        "react": "ReAct is an agent pattern..."
    }
    query = query.lower()
    for k, v in knowledge.items():
        if k in query:
            return v
    return "No information found."
"""

# ─── State Definition ─────────────────────────────────────────────────────
class PlanExecuteState(TypedDict):
    task: str
    plan: List[str]
    plan_index: int
    results: Annotated[list[str], operator.add]
    error: str                    # last execution error
    replanned: bool


# ─── Nodes ────────────────────────────────────────────────────────────────
def planner(state: PlanExecuteState):
    """Creates or improves the plan based on previous failure (if any)"""
    previous_plan_text = "\n".join(state.get("plan", ["(no previous plan)"]))
    error_info = state.get("error", "First planning attempt")

    prompt = f"""You are an expert task planner.
Original task: {state['task']}

Previous attempt failed because: {error_info}

Previous plan was:
{previous_plan_text}

Create a BETTER, more robust plan now.
Break the task into clear, small, achievable steps.
Number them 1., 2., 3., etc.
Return ONLY the numbered list of steps, nothing else."""

    response = llm.invoke([HumanMessage(content=prompt)])
    raw = response.content.strip()

    # Simple parsing of numbered steps
    steps = []
    for line in raw.split("\n"):
        line = line.strip()
        if line and (line[0].isdigit() or line.startswith("-") or line.startswith("*")):
            cleaned = line.lstrip("0123456789.-* ").strip()
            if cleaned:
                steps.append(cleaned)

    print(f"\nNew plan created ({len(steps)} steps):")
    for i, step in enumerate(steps, 1):
        print(f"  {i}. {step}")

    return {
        "plan": steps,
        "plan_index": 0,
        "results": [],           # reset results when replanning
        "error": "",
        "replanned": bool(state.get("plan"))  # mark as replanned if we had old plan
    }


def executor(state: PlanExecuteState):
    """Executes current step in the plan"""
    if state["plan_index"] >= len(state["plan"]):
        return {"plan_index": state["plan_index"]} 

    current_step = state["plan"][state["plan_index"]]
    print(f"Executing step {state['plan_index']+1}/{len(state['plan'])}: {current_step}")

    try:
        step_lower = current_step.lower()
        if any(word in step_lower for word in ["calculate", "math", "+", "-", "*", "/"]):
            result = calculator(current_step)
        else:
            result = search(current_step)

        return {
            "results": [f"Step {state['plan_index']+1}: {result}"],
            "plan_index": state["plan_index"] + 1,
            "error": ""
        }

    except Exception as e:
        error_msg = f"Execution failed: {str(e)}"
        print(f"✗ {error_msg}")
        return {"error": error_msg}  


def should_replan_or_continue(state: PlanExecuteState):
    """Routing: replan on error (once), continue, or finish"""
    if state["error"] and not state.get("replanned", False):
        print("  → Failure detected → triggering replan...\n")
        return "planner"

    if state["plan_index"] >= len(state["plan"]):
        print("  ✓ Plan completed!\n")
        return "END"

    return "executor"


# ─── Build Graph ──────────────────────────────────────────────────────────
pe_builder = StateGraph(PlanExecuteState)

pe_builder.add_node("planner", planner)
pe_builder.add_node("executor", executor)

pe_builder.add_edge(START, "planner")
pe_builder.add_conditional_edges(
    "executor",
    should_replan_or_continue,
    {"planner": "planner", "executor": "executor", "END": END}
)

plan_execute_with_replan = pe_builder.compile()


# ─── Nice Test Function ───────────────────────────────────────────────────
def test_plan_execute(task: str, thread_id="plan_test"):
    print("\n" + "═" * 80)
    print(f"TASK: {task}")
    print("═" * 80 + "\n")

    initial_state = {
        "task": task,
        "plan": [],
        "plan_index": 0,
        "results": [],
        "error": "",
        "replanned": False
    }

    result = plan_execute_with_replan.invoke(
        initial_state,
        config={"configurable": {"thread_id": thread_id}}
    )

    print("\nFINAL RESULTS:\n")
    print("Final plan:")
    for i, step in enumerate(result["plan"], 1):
        print(f"  {i}. {step}")

    print("\nExecution results:")
    for res in result["results"]:
        print(f"  • {res}")

    if result["error"]:
        print(f"\nLast error: {result['error']}")

    print("\n" + "═" * 80 + "\n")


# ─── Run some interesting test examples ───────────────────────────────────
print("Running test examples...\n")

test_plan_execute("Calculate 10 / 0 and then explain what happened")
test_plan_execute("What is Python and who created it?")
test_plan_execute("Calculate 7 factorial (7!) then multiply by 12")

Running test examples...


════════════════════════════════════════════════════════════════════════════════
TASK: Calculate 10 / 0 and then explain what happened
════════════════════════════════════════════════════════════════════════════════


New plan created (7 steps):
  1. Identify the mathematical operation to be performed (division).
  2. Define the numbers involved in the operation (numerator: 10, denominator: 0).
  3. Attempt to perform the division operation (10 divided by 0).
  4. Analyze the result of the division operation.
  5. Research the mathematical rules regarding division by zero.
  6. Summarize the findings on why division by zero is undefined.
  7. Prepare an explanation of the results and the reasoning behind them.

FINAL RESULTS:

Final plan:
  1. Identify the mathematical operation to be performed (division).
  2. Define the numbers involved in the operation (numerator: 10, denominator: 0).
  3. Attempt to perform the division operation (10 divided by 0).
  4. A

## Exercise 3: Adaptive Reflection with Quality Metrics

**Task:** Improve Reflection with numerical quality scoring.

**Requirements:**
1. Critic scores draft on multiple criteria (1-5 each):
   - Clarity
   - Completeness
   - Accuracy
2. Use Pydantic model for structured scoring
3. Only refine if any score < 4
4. Track score improvements across iterations
5. Stop when all scores ≥ 4 or max iterations

**Example:**
```
Iteration 1: Clarity=3, Completeness=4, Accuracy=5 → Refine
Iteration 2: Clarity=5, Completeness=4, Accuracy=5 → Approved!
```

**Deliverables:**
- Reflection agent with quality metrics
- Test with task requiring refinement
- Visualize score improvements

---

In [11]:
from typing import Literal, TypedDict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage

# Assuming 'llm' is already defined earlier (ChatOpenAI instance)

class QualityScore(BaseModel):
    clarity: int = Field(..., ge=1, le=5, description="1=very unclear, 5=crystal clear")
    completeness: int = Field(..., ge=1, le=5)
    accuracy: int = Field(..., ge=1, le=5)
    overall: int = Field(..., ge=1, le=5)

class ReflectionStateEx3(TypedDict):
    task: str
    draft: str
    critique: str
    scores: QualityScore | None
    iterations: int

def generator_ex3(state: ReflectionStateEx3):
    if state["iterations"] == 0:
        prompt = f"Write the best possible, high-quality answer for:\n\n{state['task']}"
    else:
        prompt = f"""Improve the previous draft significantly.

Task: {state['task']}

Previous draft:
{state['draft']}

Previous critique:
{state['critique']}

Previous scores:
{state['scores'].model_dump_json(indent=2) if state['scores'] else '(first try)'}

Make the new version clearly better in clarity, completeness and/or accuracy."""

    draft = llm.invoke([HumanMessage(content=prompt)]).content.strip()
    print(f"Generated draft (iteration {state['iterations']+1})")
    return {"draft": draft, "iterations": state["iterations"] + 1}

def critic_ex3(state: ReflectionStateEx3):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an extremely strict but fair quality evaluator.
Score each criterion 1-5 (1=poor, 5=excellent).
Be honest — 5 is only for near-perfect work.
Return ONLY valid JSON with fields: clarity, completeness, accuracy, overall"""),
        ("human", """Task: {task}

Current draft:
{draft}

Evaluate carefully and return valid JSON only.""")
    ])

    chain = prompt | llm.with_structured_output(QualityScore)
    scores: QualityScore = chain.invoke({"task": state["task"], "draft": state["draft"]})

    # Generate short human-readable critique
    critique_prompt = f"Very briefly explain your scores (2-4 sentences max):\n\n{scores.model_dump_json(indent=2)}"
    critique = llm.invoke([HumanMessage(content=critique_prompt)]).content.strip()

    print(f"Scores → C:{scores.clarity} | Comp:{scores.completeness} | Acc:{scores.accuracy} | O:{scores.overall}")

    return {
        "scores": scores,
        "critique": critique
    }

# ─── Fixed routing function ──────────────────────────────────────────────
def should_continue_ex3(state: ReflectionStateEx3) -> Literal["generator", "__end__"]:
    if state["iterations"] >= 4:
        print("Max iterations (4) reached")
        return "__end__"

    if state["scores"] is None:
        return "generator"

    s = state["scores"]
    min_score = min(s.clarity, s.completeness, s.accuracy, s.overall)

    if min_score >= 4:
        print("Quality gate PASSED (all ≥ 4) ✓")
        return "__end__"

    print(f"→ Needs improvement (lowest score = {min_score})")
    return "generator"

# ─── Build graph ─────────────────────────────────────────────────────────
refl_ex3_builder = StateGraph(ReflectionStateEx3)

refl_ex3_builder.add_node("generator", generator_ex3)
refl_ex3_builder.add_node("critic", critic_ex3)

refl_ex3_builder.add_edge(START, "generator")
refl_ex3_builder.add_edge("generator", "critic")
refl_ex3_builder.add_conditional_edges("critic", should_continue_ex3)

reflection_with_scores = refl_ex3_builder.compile()

print("Reflection agent with structured quality scoring created successfully ✓")

Reflection agent with structured quality scoring created successfully ✓


In [ ]:
# Quick test function

def test_reflection(task: str):
    initial_state = {
        "task": task,
        "draft": "",
        "critique": "",
        "scores": None,
        "iterations": 0
    }

    print(f"\n{'═'*80}")
    print(f"REFLECTION TEST: {task}")
    print("═"*80 + "\n")

    result = reflection_with_scores.invoke(initial_state)

    print("FINAL OUTPUT:")
    print(result["draft"])
    print(f"\nIterations performed: {result['iterations']}")
    if result.get("scores"):
        print("Final scores:")
        print(result["scores"].model_dump_json(indent=2))
    print("═"*80 + "\n")


# Run some tests
test_reflection("Explain what is ReAct pattern in 3-4 sentences")
test_reflection("Write a short poem about the city of Lagos at night")


════════════════════════════════════════════════════════════════════════════════
REFLECTION TEST: Explain what is ReAct pattern in 3-4 sentences
════════════════════════════════════════════════════════════════════════════════

Generated draft (iteration 1)
Scores → C:5 | Comp:5 | Acc:5 | O:5
Quality gate PASSED (all ≥ 4) ✓
FINAL OUTPUT:
The ReAct pattern, short for "Reasoning and Acting," is a framework designed to enhance the decision-making capabilities of AI systems, particularly in interactive contexts. It combines reasoning, where the AI evaluates information and generates insights, with acting, where it executes actions based on those insights. This pattern enables systems to respond dynamically to changing environments and user inputs, making them more effective in tasks such as conversational agents or complex problem-solving scenarios. By integrating reasoning with actionable outputs, the ReAct pattern fosters a more coherent and adaptive interaction between AI and users.

It